# Clinic Scheduling RL - Full Documentation

This document explains the design, code, and usage of the clinic scheduling reinforcement learning notebook. It covers environment rules, the RL setup, training, evaluation, and the 7-minute booking planner. At the end, you’ll find instructions to export this as a PDF in Google Colab.

## Overview
- Problem: Schedule patients Mon–Sat, 08:00–12:00 & 13:00–16:00, 60 scheduled/day, walk-ins until cutoff, lunch closed, handle late/no-show with recall priority.
- Approach: Custom Gymnasium environments, trained with PPO (and MaskablePPO for action masking in Env V2), plus a 7-minute slot booking planner.
- Outputs: Trained models, evaluation metrics/plots, and a booking demo.

## Installation (Colab)
Run in Colab before imports:

```bash
!pip -q install gymnasium==0.29.1 stable-baselines3==2.3.2 sb3-contrib==2.3.2 shimmy==1.3.0 plotly==5.24.1 ipywidgets
```

If packages install but imports fail, restart runtime then re-run.

## Environment V1: `ClinicSchedulingEnv`
### Purpose
A simple simulator with a single server, fixed service time (slot_minutes), lunch closure, late/no-show, walk-ins.

### Key rules
- Mon–Sat open, Sun closed (enforced at reset).
- Hours: 08–12 and 13–16; lunch 12–13.
- Max 60 scheduled slots/day; walk-ins arrive via Poisson process until cutoff.
- Late patients moved to late list; admin can recall (priority).

### State (`observation_space`)
- current "slot" index, scheduled remaining, walk-in queue length, late list length, next scheduled on-site flag, time-to-next-arrival estimate.

### Actions (`action_space`)
- 0: serve next scheduled on-site
- 1: serve walk-in
- 2: recall late patient

### Reward
- +1 per patient served, +0.05 bonus for scheduled/recall
- -0.01 idle minute when queues exist
- -0.001 × late_list size each step
- -0.1 × remaining scheduled at end of day

### Important methods
- `_maybe_generate_walkins()`: Poisson arrivals per minute
- `_update_late_status()`: marks scheduled as late after their slot if not arrived
- `step(action)`: serves patient or idles, advances time respecting lunch
- `served_log`: captures served events and wait times

## Environment V2: `ClinicSchedulingEnvV2`
### Purpose
Adds multi-provider parallel service, variable service durations, and action masking. Can seed scheduled arrivals from the booking planner.

### Additions over V1
- **Multi-provider**: `num_providers` servers; each minute decrements busy timers.
- **Variable service**: `_service_duration()` draws around mean (approx lognormal).
- **Action masking**: mask invalid actions if no eligible patient or no free providers.
- **Seeding**: `seeded_schedule=[(patient_id, start_minute), ...]` to reflect booked times.

### Observation
- minute slot, walk-in queue length, late list length, scheduled remaining, free providers, and 3 mask bits.

### Actions
Same as V1 (0 scheduled, 1 walk-in, 2 late recall) but validated by mask.

### Reward
Similar to V1; scaled by number served in a minute; end-of-day penalty for unserved scheduled.

### Logs
`served_log` records id, served minute, arrival time, walk-in flag, late flag, wait time.

## 7-minute Booking Planner
### Purpose
Expose a user-facing booking interface at 7-minute granularity, with hour labels and capacities.

### Behavior
- Hours open: 8, 9, 10, 11, 13, 14, 15.
- Hour capacity with 7-min slots: typically 9 except boundary hours (11, 15) limited to 8 (finish by lunch/close).
- After filling 8:00 with 9 bookings (patients 1..9), availability becomes 9–12 and 13–16 for the next patient.

### API
- `BookingPlanner.book(hour) -> 'h:mmam/pm' | None`
- `BookingPlanner.available_hours() -> List[int]`
- `BookingPlanner.availability_label() -> str`

### Integration
Use its output (`(patient_id, start_minute)`) to seed `ClinicSchedulingEnvV2`.

## Training
### V1
- PPO with vectorized env wrapper; default 200k steps.

### V2
- MaskablePPO if available, otherwise PPO.
- Calibration via `CALIB` dict for providers, service mean/sigma, walk-in rate, no-show/late, cutoff.
- Optional EvalCallback (disabled by default).

## Evaluation
- Summaries: served totals, breakdown scheduled/walk-in, late recalls, average waits, remaining queues.
- Charts: queues over time, wait distribution.
- V2 evaluation repeats episodes and aggregates metrics.

## Assumptions & Limitations
- Sim parameters are not data-calibrated; fit to your historical logs for realism.
- No cancellations/reschedules, triage priorities, or fairness metrics.
- Single clinic location; no room/equipment constraints modeled.
- Safety: add action masks/guards to enforce hard constraints in production.

## How to export this document to PDF in Colab
1. Open this doc notebook in Colab: `docs/clinic_scheduling_rl_docs.ipynb`.
2. File → Print → Destination: Save as PDF → Save.
   - Or: File → Download → Download as PDF (if available).
3. If you prefer programmatic export, install `nbconvert` and run:

```bash
!pip -q install nbconvert[webpdf]==7.16.4 pyppeteer==2.0.0
!jupyter nbconvert --to webpdf --allow-chromium-download docs/clinic_scheduling_rl_docs.ipynb
```

This produces `docs/clinic_scheduling_rl_docs.pdf` to download.